In [337]:
import requests 
import bs4
import pandas as pd
import numpy as np
import json
import matplotlib 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [338]:
EDIBLE_FRUITS_URL = 'https://en.wikipedia.org/wiki/List_of_culinary_fruits'
WIKIPEDIA_API_URL = 'https://en.wikipedia.org/w/api.php'

In [339]:
response = requests.get(EDIBLE_FRUITS_URL)
if response.status_code == 200:
    print ("Get Successfull")
else:
    print ("Get Failed")

Get Successfull


In [340]:
resp_soap = bs4.BeautifulSoup(response.text)

In [341]:
resp_soap.title

<title>List of culinary fruits - Wikipedia</title>

###### Header of the first table

In [342]:
resp_soap.table.tbody.tr

<tr>
<th><a href="/wiki/Common_name" title="Common name">Common name</a>
</th>
<th><a href="/wiki/Binomial_nomenclature" title="Binomial nomenclature">Species name</a>
</th>
<th><a href="/wiki/Cultivar" title="Cultivar">Cultivar list</a>
</th></tr>

In [343]:
def add_to_dictionary(table_dict):
    
    if (row_list[0]) in table_dict:
        table_dict[row_list[0]][1] = pos_dict[row_list[0]][1] + ", " + row_list[2]
    else:
        table_dict[row_list[0]] = [row_list[1],row_list[2]]

In [344]:
def process_row(row,ncolumns=0):
    
    row_values = []
    col_count = 0
    for column in row:
        if isinstance(column,bs4.element.Tag): # Avoid processing NavigableString - Mostly newline char      
            col_count += 1
            row_values.append(column.get_text(strip=True))
            
            if col_count == 1:
                if column.find('a'):
                    row_values.append(column.a.get_attribute_list('href')[0])
                else:
                    row_values.append('')
            
        if ncolumns > 0 and col_count >= ncolumns:
            break
            
    return row_values

In [345]:
def process_table(soap_msg_table,ncolumns=0):
    
    row_list = []
    for row in soap_msg_table:
        
        if isinstance(row,bs4.element.Tag): # Avoid processing NavigableString - Mostly newline char
            row_list.append(process_row(row,ncolumns))
            
    #print (row_list)
    return row_list

In [346]:
all_edible_fruits = pd.DataFrame()
for table in resp_soap.find_all("table"):
    common_spices = pd.DataFrame(process_table(table.tbody,ncolumns=3))
    header = common_spices.loc[0]
    common_spices = common_spices[1:]
    all_edible_fruits = all_edible_fruits.append(common_spices)
    
all_edible_fruits.columns = ['Common name','Page','Species name','Cultivar list']

In [347]:
all_edible_fruits.head()

,Common name,Page,Species name,Cultivar list
1,Apple,/wiki/Apple,Malus pumila,Apple cultivars
2,Chinese quince,/wiki/Chinese_quince,Pseudocydonia sinensis,
3,Chokeberry,/wiki/Chokeberry,Aronia melanocarpa,
4,Cocky apple,/wiki/Cocky_apple,Planchonia careya,
5,Eastern mayhaw,/wiki/Crataegus_aestivalis,Crataegus aestivalis,


In [348]:
all_edible_fruits.shape

(422, 4)

In [349]:
all_edible_fruits.loc[all_edible_fruits.index[1]].head()

,Common name,Page,Species name,Cultivar list
2,Chinese quince,/wiki/Chinese_quince,Pseudocydonia sinensis,
2,Acerola,/wiki/Acerola,Malpighia emarginata,
2,Amanatsu,/wiki/Amanatsu,Citrus × natsudaidai,
2,Amazon grape,/wiki/Amazon_grape,Pourouma cecropiifolia,
2,Canary melon,/wiki/Canary_melon,Cucumis melo var. inodorus'Canary Melon',


In [350]:
all_edible_fruits = all_edible_fruits.reset_index()
all_edible_fruits.drop('index',axis=1,inplace=True)

In [351]:
all_edible_fruits.drop(all_edible_fruits.index[all_edible_fruits['Page'] == '']).shape

(419, 4)

In [352]:
all_edible_fruits.loc[all_edible_fruits['Common name'] == 'Saguaro fruit']

,Common name,Page,Species name,Cultivar list
410,Saguaro fruit,/wiki/Carnegiea_gigantea,Carnegiea gigantea,


###### Get the data using Wikipedia API

In [353]:
def get_wiki_url(end_point):
    return WIKIPEDIA_API_URL + f'?action=parse&format=json&page={end_point.split("/")[-1]}'
    #return 'https://en.wikipedia.org' + end_point

In [354]:
def get_child_pages(end_points):

    massive_text = ""
    for ep in end_points:
        url =get_wiki_url(ep)
        print(url)
        response = requests.get(url)
        if response.status_code != 200:
            print ("Get Failed") # Only report if any failures
        
        jresponse = json.loads(response.text)
        
        if 'parse' in jresponse:
            #print(jresponse['parse'].keys())
            #print(jresponse['parse']['text']['*'])
            massive_text += bs4.BeautifulSoup(jresponse['parse']['text']['*']).get_text()
    
    return massive_text

In [355]:
massive_text = get_child_pages(all_edible_fruits['Page'][1:2])

https://en.wikipedia.org/w/api.php?action=parse&format=json&page=Chinese_quince


```
stopwords = set(STOPWORDS)
# instantiate a word cloud object
response_wc = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
)

# generate the word cloud
response_wc.generate(massive_text)```

```
fig = plt.figure()
fig.set_figwidth(14)
fig.set_figheight(18)
plt.imshow(response_wc, interpolation='bilinear')

plt.axis('off')
plt.show()
```